# Prismatic hopper

In [ ]:
import sympy as sp, numpy as np
from sympy import Matrix as Mat
vec = lambda *args: sp.Matrix(args)

%load_ext autoreload
%autoreload 2
# import ckc_lib
import physical_education as pe

In [ ]:
constants = mb, ml, rb, rl, g = sp.symbols('m_b m_l r_b r_l g', real=True)
constants_mapping = dict(zip(constants, (4.1, 0.090 * 4, 0.15, (0.1375+0.250)*2/2, 9.81)))

Ib, Il = mb*(rb**2 + rb**2)/12, (ml*rl**2)/12
Ib = 0

# two input torques
Fr, Fth = sp.Matrix(sp.symbols('F_r F_th', real=True))
u = vec(Fr, Fth)

# two ground reaction forces
Lx, Ly = sp.symbols('L_x L_y', real=True)

In [ ]:
# the states of the leg
def sym_and_derivs(symbol: str):
    return sp.symbols(r'%s \dot{%s} \ddot{%s}' % (symbol, symbol, symbol))

x, dx, ddx = sym_and_derivs('x')
y, dy, ddy = sym_and_derivs('y')
r, dr, ddr = sym_and_derivs('r')

thb, dthb, ddthb = sym_and_derivs('\\theta_b')
thl, dthl, ddthl = sym_and_derivs('\\theta_l')

q   = Mat([  x,   y,   r,   thb,   thl])
dq  = Mat([ dx,  dy,  dr,  dthb,  dthl])
ddq = Mat([ddx, ddy, ddr, ddthb, ddthl])
# B   = Mat([  0,   0,   Fth,  -Fth, -τr])

# q, dq, ddq, B

In [ ]:
# the base
Pb_I = Mat([x, y])
Rb_I = pe.utils.rot(thb).T

# connection between base and top links
conn_b = Pb_I

# the top links (shoulder to knee)
leg_len = rl + r
Pl_l = Mat([0, - leg_len/2])
Rl_I = pe.utils.rot(thl).T
Pl_I = sp.simplify(conn_b + Rl_I @ Pl_l)

foot = sp.simplify(conn_b + Rl_I @ Mat([0, -leg_len]))

# Pb_I, Pl_I, foot

In [ ]:
# calculate the system's kinetic and potential energy
Ps = [Pb_I, Pl_I]
Rs = [Rb_I, Rl_I]
dths = dq[3:]
Is = [Ib, Il]
ms = [mb, ml]

dxy = [P.jacobian(q) @ dq for P in Ps]

Ek = sp.simplify(sum(
    0.5*m*(dx**2 + dy**2) + 0.5*I*dth**2
        for m, I, (dx, dy), dth in zip(ms, Is, dxy, dths)
))

Ep = sp.simplify(sum(
    m*g*y for m, (_, y) in zip(ms, Ps)
))

# Ek, Ep

In [ ]:
# manipulator equation parts: mass matrix, coriolis, gravity/NC force
M, C, G = pe.utils.manipulator_equation(Mat([Ek]), Mat([Ep]), q, dq)

M = sp.simplify(M); G = sp.simplify(G)

In [ ]:
J_L = foot.jacobian(q)
L = Mat([Lx, Ly])

In [ ]:
def getb():
    fr = Rl_I @ vec(0, -Fr)
    rr = Pl_I
    Qr = (fr.T @ rr.jacobian(q)).T
    
    fth = Mat([Fth])
    rth = Mat([thb-thl])
    Qth = (fth.T @ rth.jacobian(q)).T
    
    return Qr + Qth

B = getb().jacobian(u)

In [ ]:
EOM = sp.simplify((
    M @ ddq + C + G - B @ u - J_L.T @ L
).subs(constants_mapping))